# Description
In this experiment I will be writing a Convnet that can recognize/differentiate between two types of pictures, with cats and dogs. As an added challenge, I will be working on a small pictures subset - just 1000 for each class. So generating new examples will be required. As the last step I will use a pre-trained convolutional network to prepare the final network for picture recognition. 

Whole experiment is based on code from "Deep Learning with Python" book.

## 1. Preparing data for experiment